In [2]:
from bs4 import BeautifulSoup
import re
import time
import requests
import random
import string

import pandas as pd


In [3]:
def clean_text(text: str) -> str:
    txt = text.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    txt = txt.replace('  ', ' ')
    return txt.strip()

def extract_by_regex(text: str, regex: str) -> str:
    pattern = re.compile(regex)
    match = pattern.search(text)
    if match:
        # Check if there are any groups and return the first group if it exists
        if match.groups():
            return match.group(1) + " " + match.group(2) if len(match.groups()) > 1 else match.group(1)
        else:
            return match.group(0)  # Return the entire match if no groups are defined
    return ""

In [7]:
class TripAdvisorScraper:
    def __init__(self, url):
        self.url = url
        self.soup = None


    def fetch_page(self):
        
        # Set headers
        random_request_id = "".join(
                random.choice(string.ascii_lowercase + string.digits) for i in range(180)
            )
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
            'accept-language': 'en-US,en;q=0.9,fr;q=0.8',
            "X-Requested-By": random_request_id,
            "Referer": "https://www.tripadvisor.com/Hotels",
            "Origin": "https://www.tripadvisor.com",
            'accept-encoding': 'gzip, deflate, br',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
            'encoding': 'utf-8',
        }
        
        # Send a GET request
        response = requests.get(self.url, headers=headers)
        try:
            # Check encoding
            
            if response.headers.get('Content-Encoding') == 'gzip':
                content = response.content.decode('gzip')
            elif response.headers.get('Content-Encoding') == 'deflate':
                content = response.content.decode('zlib')
            # elif response.headers.get('Content-Encoding') == 'br':
            #     content = brotli.decompress(response.content)
            else:
                content = response.text 
        
        except Exception as e:
            print(f"Error: {e}")
            content = response.text
        finally:    
            self.soup = BeautifulSoup(content, 'html.parser')

    def print_soup(self):
        if self.soup:
            print(self.soup.prettify())
        else:
            print("Soup is not initialized. Please fetch the page first.")



    

In [ ]:
class TripAdvisorRestaurantsScrapper(TripAdvisorScraper):
    def __init__(self, url):
        super().__init__(url)
        self.restaurant_data = []
        
    def get_review_cards(self):
        if self.soup:
            review_cards = self.soup.find_all('div', class_='_c', attrs={'data-automation': 'reviewCard'})
            return review_cards
        else:
            print("Soup is not initialized. Please fetch the page first.")
            return []
        
    def get_next_url(self, base_url):
        next_url = self.soup.find('a', class_='BrOJk u j z _F wSSLS tIqAi unMkR', attrs={'aria-label':"Next page"})
        if next_url is not None:
            next_url = base_url + next_url.get('href')
            return next_url
        else:
            return None
        


    def extract_restaurant_data(self):
        if not self.soup:
            self.fetch_page()
        
        # Extract the main container
        main_container = self.soup.find("div", class_="_1kXteagE

In [6]:
base_url = "https://www.tripadvisor.com"
restaurants_comp = "/FindRestaurants?geo=187265&offset=0&establishmentTypes=10591&minimumTravelerRating=TRAVELER_RATING_LOW&broadened=false"
full_url = base_url + restaurants_comp
full_url

'https://www.tripadvisor.com/FindRestaurants?geo=187265&offset=0&establishmentTypes=10591&minimumTravelerRating=TRAVELER_RATING_LOW&broadened=false'